In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten

from encoder import Encoder

In [8]:
config = {'num_heads': 12, 
          'vocab_size': 30522,
          'hidden_size': 128,
          'max_position_embeds': 512,
          'intermediate_size': 512,
          'dropout_p': 0.1,
          'input_size': (100,),
          'num_hidden_layers': 1}

## Build a classifier on our transformer encoder

In [9]:
clf = Sequential([InputLayer(input_shape=config['input_size']),
                  Encoder(config),
                  Flatten(),
                  Dense(config['vocab_size'], activation='softmax')])

clf.build()
clf.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoder (Encoder)           (None, 100, 128)          4166760   
                                                                 
 flatten_3 (Flatten)         (None, 12800)             0         
                                                                 
 dense_159 (Dense)           (None, 30522)             390712122 
                                                                 
Total params: 394,878,882
Trainable params: 394,878,882
Non-trainable params: 0
_________________________________________________________________
